# Анализ базы данных через SQL

Мы компания, купившая крупный сервис для чтения книг по подписке.
<br>Цель исследования: сформулировать ценностное предложение для нового продукта.
<br> В качестве инструмента для анализа у нас есть база данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.

<br>План исследования:
1. Знакомство с данными


2. Решение задач через SQL-запросы с выводами 
- кол-во книг, вышедших после 1 января 2000 года
- кол-во обзоров и средняя оценка для каждой книги
- определить издательство, которое выпустило наибольшее число книг (толще 50 страниц)
- определить автора с самой высокой средней оценкой книг (оценка 50 и больше)
- посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
3. Общий вывод

## Знакомство с данными

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
books_data = pd.io.sql.read_sql(sql=text(query), con=con)
books_count = pd.io.sql.read_sql(sql='SELECT COUNT(*) as total FROM books', con=con)
display(books_data)
print(f'Количество строк: {books_count["total"][0]}')

# Для authors
query_authors = '''SELECT * FROM authors LIMIT 5'''
authors_data = pd.io.sql.read_sql(sql=text(query_authors), con=con)
author_count = pd.io.sql.read_sql(sql='SELECT COUNT(*) as total FROM authors', con=con)
display(authors_data)
print(f'Количество строк: {author_count["total"][0]}')

# Для publishers
query_publishers = '''SELECT * FROM publishers LIMIT 5'''
publishers_data = pd.io.sql.read_sql(sql=text(query_publishers), con=con)
publisher_count = pd.io.sql.read_sql(sql='SELECT COUNT(*) as total FROM publishers', con=con)
display(publishers_data)
print(f'Количество строк: {publisher_count["total"][0]}')

# Для ratings
query_ratings = '''SELECT * FROM ratings LIMIT 5'''
ratings_data = pd.io.sql.read_sql(sql=text(query_ratings), con=con)
rating_count = pd.io.sql.read_sql(sql='SELECT COUNT(*) as total FROM ratings', con=con)
display(ratings_data)
print(f'Количество строк: {rating_count["total"][0]}')

# Для reviews
query_reviews = '''SELECT * FROM reviews LIMIT 5'''
reviews_data = pd.io.sql.read_sql(sql=text(query_reviews), con=con)
review_count = pd.io.sql.read_sql(sql='SELECT COUNT(*) as total FROM reviews', con=con)
display(reviews_data)
print(f'Количество строк: {review_count["total"][0]}')

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Количество строк: 1000


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Количество строк: 636


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Количество строк: 340


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Количество строк: 6456


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Количество строк: 2793


ВЫВОД ПО РАЗДЕЛУ: В базе данных 5 таблиц. Самая многострочная с оценками книг. 

## Решение задач через SQL-запросы с выводами

### Кол-во книг, вышедших после 1 января 2000 года

In [2]:
# Запрос для подсчета книг после 1 января 2000 года
count_query = '''SELECT COUNT(*) as book_id FROM books WHERE publication_date > '2000-01-01' '''
books_count = pd.io.sql.read_sql(sql=text(count_query), con=con)

# Отображаем данные
print(f'Количество книг после 1 января 2000 года: {books_count["book_id"][0]}')

Количество книг после 1 января 2000 года: 819


Из 1000 книг 819 вышли после 1 января 2000 года.

### Кол-во обзоров и средняя оценка для каждой книги

In [3]:
count_book = '''SELECT b.title, 
    COUNT(DISTINCT re.review_id) AS total_reviews, 
    AVG(ra.rating) AS average_rating
FROM books b
LEFT JOIN reviews re ON b.book_id=re.book_id
LEFT JOIN ratings ra ON b.book_id=ra.book_id
GROUP BY b.book_id, b.title
ORDER BY total_reviews DESC
'''
con = engine.connect()
book_reviews_data = pd.io.sql.read_sql(sql=text(count_book), con=con)

# Отображаем данные
display(book_reviews_data)


,title,total_reviews,average_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


У книги Twilight (Twilight #1) больше всего обзоров - 7, при этом ее средняя оценка - 3.66. У книг, у которых обзоров не было встречаются разные оценки, в т.ч. 5. 

### Определить издательство, которое выпустило наибольшее число книг (толще 50 страниц)

In [4]:
count_publisher = '''SELECT COUNT(*), p.publisher
FROM publishers p
JOIN books b ON p.publisher_id=b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY count(*) DESC
LIMIT 1
'''
con = engine.connect()
books_publishers = pd.io.sql.read_sql(sql=text(count_publisher), con=con)

# Отображаем данные
display(books_publishers)

,count,publisher
0,42,Penguin Books


Издательство Penguin Books выпустило больше остальных книг - 42 шт. 

### Определить автора с самой высокой средней оценкой книг (оценка 50 и больше)

In [5]:
avg_author = '''SELECT a.author,
                       AVG(r.rating) AS avg_rating
FROM authors a
JOIN books b ON b.author_id=a.author_id
JOIN ratings r ON b.book_id=r.book_id
WHERE b.book_id IN (
    SELECT b.book_id
    FROM books b
    JOIN ratings r ON b.book_id=r.book_id
    GROUP BY b.book_id
    HAVING COUNT(r.rating) >= 50
)
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1
'''
con = engine.connect()
author_rating = pd.io.sql.read_sql(sql=text(avg_author), con=con)

# Отображаем данные
display(author_rating)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Выше средняя оценка оказалась у автора J.K. Rowling/Mary GrandPré - 4.29.

### Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [6]:
avg_review = '''SELECT AVG(re.review_id) avg_review
FROM reviews re
JOIN books b ON b.book_id=re.book_id
JOIN ratings ra ON b.book_id=ra.book_id
HAVING COUNT(ra.rating_id) > 48
ORDER BY avg_review DESC
'''
con = engine.connect()
reviews_rating = pd.io.sql.read_sql(sql=text(avg_review), con=con)

# Отображаем данные
display(reviews_rating)

,avg_review
0,1441.602782


In [7]:
avg_review = '''SELECT AVG(reviews_cnt) AS avg_review       
            
                  FROM (
                       SELECT username,
                              COUNT(review_id) AS reviews_cnt
                          FROM reviews
                      GROUP BY username
                        HAVING username IN (
                                        SELECT username
                                        FROM ratings
                                        GROUP BY username
                                        HAVING COUNT(rating_id) > 48   
                                      ) 
                 ) AS reviews_avg '''
con = engine.connect()
reviews_rating = pd.io.sql.read_sql(sql=text(avg_review), con=con)

# Отображаем данные
display(reviews_rating)

,avg_review
0,24.0


У пользователей, которые поставили больше 48 оценок в среднем написано по 24 обзора. 

## Общий вывод

В разделе ЗНАКОМСТВО С ДАННЫМИ мы познакомились с 5-ю таблицами. Самая многострочная среди них таблица с оценками книг.
<br>Изучив детально данные и посчитав некоторые параметры наша компания может представить новый сервис для чтения книг по подписке, который предлагает уникальные возможности и преимущественно удовлетворяет потребности современных читателей. Вот ключевые выводы и ценности, которые мы выделили:

1. Широкий выбор современного контента: 
   Из 1000 книг 819 были выпущены после 1 января 2000 года, что подтверждает наш фокус на актуальных и современных произведениях, соответствующих интересам пользователей.

2. Популярные бестселлеры с активным обсуждением: 
   Книга Twilight (Twilight #1) выделяется своим количеством обзоров — 7. Несмотря на среднюю оценку 3.66, это свидетельствует о том, что пользователи активно обсуждают контент, и дают возможность новым читателям легко находить самые обсуждаемые произведения.

3. Разнообразие мнений и оценок: 
   В нашем каталоге есть книги, на которые еще не написаны обзоры, но среди них уже встречаются наивысшие оценки. Это говорит о том, что новинки в нашем сервисе могут стать следующими любимыми книгами для многих читателей.

4. Сильные издательства для качественного контента: 
   С издательством Penguin Books, выпустившим больше всего книг — 42, мы гарантируем наличие качественного и разнообразного контента, который соответствует высоким стандартам.

5. Лучшая литература от известных авторов: 
   Для ценителей качественной литературы, у нас в ассортименте представлены работы таких авторов, как J.K. Rowling/Mary GrandPré, имеющая среднюю оценку 4.29. Это подтверждает наличие высококачественного контента от признанных авторов.

6. Ценность отзывов: 
   Среднее количество обзоров книг, которые написали пользователи с 48 оценками и больше составляет 24 обзора. Это позволяет нашим пользователям принимать более информированные решения при выборе книг, знакомясь с мнением других читателей.

Данный сервис станет обширной платформой для любителей чтения, предоставляя доступ к огромному количеству книг, поддерживая активное взаимодействие и обмен мнениями среди читателей, а также гарантируя качественное чтение от самых известных авторов и издательств.